In [ ]:
import warnings
warnings.filterwarnings(action="ignore", module="coffea.*")

import awkward as ak
import coffea
from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema
import dask
from dask.distributed import Client
import hist.dask
import matplotlib.pyplot as plt
import mplhep
import numpy as np
import uproot

mplhep.style.use("ATLAS")
client = Client("tls://localhost:8786")

Just like in the PHYSLITE demo earlier today, we will only load a few branches, this speeds things up a bit. Add more branches here if you need them!

In [ ]:
def filter_name(name):
    return name in [
        "EventInfoAuxDyn.mcEventWeights",
        
        "AnalysisElectronsAuxDyn.pt",
        "AnalysisElectronsAuxDyn.eta",
        "AnalysisElectronsAuxDyn.phi",
        "AnalysisElectronsAuxDyn.m",
        
        "AnalysisMuonsAuxDyn.pt",
        "AnalysisMuonsAuxDyn.eta",
        "AnalysisMuonsAuxDyn.phi",

        "AnalysisJetsAuxDyn.pt",
        "AnalysisJetsAuxDyn.eta",
        "AnalysisJetsAuxDyn.phi",
        "AnalysisJetsAuxDyn.m",
        
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pb",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pc",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pu",
    ]


Load the samples into `NanoEventsFactory` to analyze their content.

In [ ]:
# using ATLAS Open Data
samples = {
    "sample 1": "mc20_13TeV/DAOD_PHYSLITE.37621317._000001.pool.root.1",
    "sample 2": "mc20_13TeV/DAOD_PHYSLITE.37620644._000012.pool.root.1",
    "sample 3": "mc20_13TeV/DAOD_PHYSLITE.37865929._000022.pool.root.1",
    "sample 4": "mc20_13TeV/DAOD_PHYSLITE.37621204._000012.pool.root.1",
    "sample 5": "mc20_13TeV/DAOD_PHYSLITE.38191712._000013.pool.root.1",
    "sample 6": "mc20_13TeV/DAOD_PHYSLITE.38191575._000015.pool.root.1"
}
prepend_path = "root://xcache.af.uchicago.edu:1094//root://eospublic.cern.ch//eos/opendata/atlas/rucio/"

events_list = [
    NanoEventsFactory.from_root(
        {prepend_path + samples[sample]: "CollectionTree"},
        delayed=True,
        schemaclass=PHYSLITESchema,
        uproot_options={"filter_name": filter_name}
    ).events()
    for sample in samples.keys()
]

# as introduced by Jim earlier
EVENT = 0
PARTICLE = 1

Your task: map all of these samples to the physics process they correspond to! They contain to the following processes:

- `HWW`: gluon fusion $H\rightarrow WW^* \rightarrow \ell \nu \ell \nu$
- `HZZ`: gluon fusion $H\rightarrow ZZ^* \rightarrow \ell \ell \ell \ell$
- `tchan`: single top production in $t$-channel
- `ttbar`: $t\bar{t}$ production with at least one light lepton in the final state
- `tZq`: associated production of single top quark and $Z$ boson
- `Zee`: $Z\rightarrow ee$ Drell–Yan production,

You should solve this using kinematic information in the events. Your solution will be a map of samples (`sample 1` etc.) to these processes (e.g. `Zee`).

To get us started, let's look at the distribution of b-tagged jets in the samples. We can use DL1d and calculate the discriminant ourselves from the information in the PHYSLITE files. Relevant information is at https://ftag.docs.cern.ch/recommendations/algs/r22-preliminary/#working-point-definition-for-dl1dv01 with the discriminant defined at https://ftag.docs.cern.ch/recommendations/algs/2019-recommendations/#algorithm-structure.

In [ ]:
h = hist.dask.Hist.new.Regular(5, 0, 5, name="nBtags", label="number of b-tags@70%").\
                       StrCat([], growth=True, name="samplename").Weight()

for events, name in zip(events_list, samples.keys()):
    jets = events.Jets
    jets["DL1dv01_pb"] = events.BTagging_AntiKt4EMPFlow.DL1dv01_pb
    jets["DL1dv01_pc"] = events.BTagging_AntiKt4EMPFlow.DL1dv01_pc
    jets["DL1dv01_pu"] = events.BTagging_AntiKt4EMPFlow.DL1dv01_pu

    jets = jets[jets.pt > 25_000]  # 25 GeV pT cut

    f_c = 0.018
    BTAG_CUT = 3.493  # 70% efficiency cut
    dl1d = np.log(jets["DL1dv01_pb"] / (f_c * jets["DL1dv01_pc"] + (1-f_c) * jets["DL1dv01_pu"]))
    nbtags = ak.sum(dl1d > BTAG_CUT, axis=PARTICLE)

    h.fill(nBtags=nbtags, samplename=name, weight=ak.num(nbtags, axis=EVENT)**(-1.0))

h_computed = h.compute()

fig, ax = plt.subplots()
h_computed.plot(ax=ax, linewidth=2)
ax.legend();